## Machine Learning 프로젝트 수행을 위한 코드 구조화

`(분류, 회귀 Task)`

- ML project를 위해서 사용하는 템플릿 코드를 만듭니다.

1. **필요한 라이브러리와 데이터를 불러옵니다.**


2. **EDA를 수행합니다.** 이 때 EDA의 목적은 풀어야하는 문제를 위해서 수행됩니다.


3. **전처리를 수행합니다.** 이 때 중요한건 **feature engineering**을 어떻게 하느냐 입니다.


4. **데이터 분할을 합니다.** 이 때 train data와 test data 간의 분포 차이가 없는지 확인합니다.


5. **학습을 진행합니다.** 어떤 모델을 사용하여 학습할지 정합니다. 성능이 잘 나오는 GBM을 추천합니다.


6. **hyper-parameter tuning을 수행합니다.** 원하는 목표 성능이 나올 때 까지 진행합니다. 검증 단계를 통해 지속적으로 **overfitting이 되지 않게 주의**하세요.


7. **최종 테스트를 진행합니다.** 데이터 분석 대회 포맷에 맞는 submission 파일을 만들어서 성능을 확인해보세요.

## 1. 라이브러리, 데이터 불러오기

In [1]:
pip install -U scikit-learn

In [2]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 모델들, 성능 평가
# (저는 일반적으로 정형데이터로 머신러닝 분석할 때는 이 2개 모델은 그냥 돌려봅니다. 특히 RF가 테스트하기 좋습니다.)
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from lightgbm.sklearn import LGBMClassifier
from lightgbm.sklearn import LGBMRegressor
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import roc_auc_score

# 상관관계 분석, VIF : 다중공선성 제거
from statsmodels.stats.outliers_influence import variance_inflation_factor

# KFold(CV), partial : optuna를 사용하기 위함
from sklearn.model_selection import KFold
from functools import partial

# hyper-parameter tuning을 위한 라이브러리, optuna
import optuna

In [3]:
# flag setting
data_reducing = False ## memory reducing technique
feature_reducing = False ## feature extraction (curse of dimensionality)

In [4]:
# 데이터를 불러옵니다.
train = pd.read_csv('../input/tabular-playground-series-aug-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-aug-2022/test.csv')
print(train.shape, test.shape)

In [5]:
train.columns

In [6]:
tr_col = ['id', 'product_code', 'loading', 'attribute_0', 'attribute_1',
       'attribute_2', 'attribute_3', 'measurement_0', 'measurement_1',
       'measurement_2', 'measurement_3', 'measurement_4', 'measurement_5',
       'measurement_6', 'measurement_7', 'measurement_8', 'measurement_9',
       'measurement_10', 'measurement_11', 'measurement_12', 'measurement_13',
       'measurement_14', 'measurement_15', 'measurement_16', 'measurement_17',
       'failure']

##EDA
1. 데이터 크기 확인
2. 결측치 체크
3. dtype이 object인 column 확인
4. target distribution --> target value에 대한 분포 확인!

- 데이터에서 찾아야 하는 기초적인 내용들을 확인합니다.


- class imbalance, target distribution, outlier, correlation을 확인합니다.

#### 결측치 처리

In [7]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)

 # Product_Code, ID 제거
train = train.drop(columns=['id','product_code'])

for i in range(len(train)): #앞에 Meterial_ 제거
  train['attribute_0'][i] = train['attribute_0'][i].split('_')[1]
  train['attribute_1'][i] = train['attribute_1'][i].split('_')[1]

# Min Max scaler
from sklearn.preprocessing import MinMaxScaler

scaled = MinMaxScaler().fit_transform(train)
train_scaled = pd.DataFrame(data=scaled, columns=train.columns, index=train.index)

# KNN Imputer
train_filled = imputer.fit_transform(train_scaled)
train = pd.DataFrame(train_filled)
train

In [8]:
#TEST 데이터 전처리
test = test.drop(columns=['id','product_code'])

for i in range(len(test)): #앞에 Meterial_ 제거
  test['attribute_0'][i] = test['attribute_0'][i].split('_')[1]
  test['attribute_1'][i] = test['attribute_1'][i].split('_')[1]

scaled = MinMaxScaler().fit_transform(test)
test_scaled = pd.DataFrame(data=scaled, columns=test.columns, index=test.index)

#KNN imputer 
test_filled = imputer.fit_transform(test_scaled)
test = pd.DataFrame(test_filled)
test

# 3. Training

In [9]:
X_train = train.drop(columns=[23])
target = train[23]

In [10]:
#모델 한번에 돌리기
models = {'xgb':XGBClassifier(random_state=42,
        max_depth=2,
        min_child_weight=14,
        gamma=13,
        n_estimators=100,),
          
           'gbc':GradientBoostingClassifier(n_estimators = 200, loss = 'deviance',learning_rate = 0.01),
           'lr':LogisticRegression(C= 0.01, penalty = 'l1', solver = 'liblinear', max_iter = 200),
          'lda': LinearDiscriminantAnalysis(),
#           'knn': KNeighborsClassifier(algorithm = 'brute', n_jobs=-1)
         }

In [11]:
from sklearn.model_selection import KFold, cross_val_score
def kf_cross_val(model,X,y):
    
    scores,feature_imp, features = [],[], []
    
    kf = KFold(n_splits=3,shuffle = True, random_state=42)
    #Group KF로 수정하기
    for fold, (train_index, test_index) in enumerate(kf.split(X, y)):
        
        x_train = X.iloc[train_index]
        y_train = y.loc[train_index]
        x_test = X.loc[test_index]
        y_test = y.loc[test_index]
        
        model.fit(x_train,y_train)
        
        y_pred = model.predict_proba(x_test)[:,1]     # edit 
        scores.append(roc_auc_score(y_test,y_pred))
        
        try:
            feature_imp.append(model.feature_importances_)
            features.append(model.feature_names_)
        except AttributeError: # if model does not have .feature_importances_ attribute
            pass
        
    return feature_imp, scores, features

In [12]:
# ====================================================
# Cross Validation results / comparing models
# ====================================================

results  = {}


for name,model in models.items():
    
    feature_imp,socres,features = kf_cross_val(model, X_train, target)
    results[name] = socres

for name, socres in results.items():
    print("----------\n" + name)
    print(np.mean(socres))
    print(np.std(socres))
    print(feature_imp)

In [13]:
clf = LinearDiscriminantAnalysis()
clf.fit(X_train, target)

In [14]:
pred = clf.predict(X_train)

print("train roc_auc_score : %.4f" % roc_auc_score(target,pred))

In [15]:
result = clf.predict(test)
result

### 4. Evaluation


- validation 데이터로 overfitting 같은 학습이 잘못되는 케이스를 확인.

In [42]:
result = LinearDiscriminantAnalysis(test)
result

In [17]:
# from sklearn.metrics import roc_auc_score

# pred_train = reg.predict(X_train)  # train data에 대한 예측값
# pred_test = reg.predict(X_test)    # test data에 대한 예측값

# print("Train MSE : %.4f" % roc_auc_score(y_train, pred_train))
# print("Test MSE : %.4f" % roc_auc_score(y_test, pred_test))

### 5. Prediction


- test 데이터 **train 데이터에 적용한 전처리 방식과 같은** 방법을 적용해주어야 합니다.

In [ ]:
# # Object Column 제거
# test = test.drop(columns=['id','product_code'])
# ## loading, measurement_3~17
# test.loading = test.loading.fillna(test.loading.mean())
# for x in measurement_list:
#   test[x] = test[x].fillna(test[x].mean())
# train
# #attribute_0~1 타입 변경
# for i in range(len(test)):
#   test['attribute_0'][i] = test['attribute_0'][i].split('_')[1]
#   test['attribute_1'][i] = test['attribute_1'][i].split('_')[1]
# # attribute_0,1 데이터 타입 변경
# test.attribute_0.astype(np.int64)
# test.attribute_1.astype(np.int64)

In [ ]:
# result = reg.predict(test)
# result

In [19]:
submission = pd.read_csv('../input/tabular-playground-series-aug-2022/sample_submission.csv')
submission

In [28]:
submission["failure"].value_counts()

In [32]:
submission["failure"].value_counts

In [ ]:
version_x = input()
submission["failure"] = result
submission.to_csv(f"../kaggle/working/submission_{version_x}.csv", index=False)